# Where to Setup Shop in The Twin Cities
### By Kevin Chou

The purpose of this notebook is to help or give potential investors or business owners initial guidance on where to open a new Asian Grocery Market.

In [ ]:
#Import neccessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
from bs4 import BeautifulSoup

## Scrape the zipatlas.com website to get the asian population data in MN

In [ ]:
urls = ['http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.1.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.2.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.3.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.4.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.5.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.6.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.7.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.8.htm',
        'http://zipatlas.com/us/mn/zip-code-comparison/percentage-asian-population.9.htm']

#loading empty array for board members
table = []
asians = []
#Loop through our URLs we loaded above
for b in urls:
    html = requests.get(b).text
    soup = BeautifulSoup(html, "html.parser")
#identify table we want to scrape
    table = soup.find('table', {'rules' : "all"})
#try clause to skip any companies with missing/empty board member tables
    try:
#loop through table, grab each of the 4 columns shown (try one of the links yourself to see the layout)
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            if len(cols) == 7:
                 asians.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip(), cols[3].text.strip(),
                         cols[4].text.strip(),cols[5].text.strip(),cols[6].text.strip()))
    except: pass  
COLUMNS = ['Nbr','Zipcodes','Location', 'city', 'Population', '% Asians', 'National Rank']
df = pd.DataFrame(asians, columns=COLUMNS).drop(0, axis=0)
df = df[df.Nbr != '#']
del df['Nbr']
df = df.reset_index(drop=True)

In [ ]:
# new data frame with split value columns 
new = df["Location"].str.split(", ", n = 1, expand = True) 
# making separate first name column from new data frame 
df["Latitude"]= new[0]  
# making separate last name column from new data frame 
df["Longitude"]= new[1]  
# Dropping old Name columns 
df.drop(columns =["Location"], inplace = True) 
new = df["city"].str.split(", ", n = 1, expand = True) 
df["City"]= new[0]    
df["State"]= new[1]    
df.drop(columns =["city"], inplace = True) 
# df display 
df.head() 

In [ ]:
#Export to file for safe keeping
df.to_csv('/Users/Mcair/Desktop/projects/DS_Code/MNAsians.csv',index=False)

In [ ]:
#Import MNAsians.csv if file already exists and not scraping
df = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/MNAsians.csv')

In [ ]:
df.dtypes

In [ ]:
df['Population'] = df['Population'].astype('str')
df['Population'] = df['Population'].str.replace(',', '')
df['Population'] = df['Population'].astype('float64')
df['Zipcodes'] = df['Zipcodes'].astype('str')

In [ ]:
df.shape

In [ ]:
#Import Twin Cities zip codes to use for filtering
tczip = pd.read_csv('TCZipCodes.csv')
tczip['TCZipCodes'] = tczip['TCZipCodes'].astype('str')

In [ ]:
#Join previous dataset with TCZipCodes to filter out zipcode not belonging to the Twin Cities
dfnew = pd.merge(tczip, df, how='inner', left_on = 'TCZipCodes', right_on = 'Zipcodes')
dfnew.drop(columns =["TCZipCodes"], inplace = True) 
dfnew.head()

In [ ]:
dfnew['Latitude'] = dfnew['Latitude'].astype('float64')
dfnew['Longitude'] = dfnew['Longitude'].astype('float64')

In [ ]:
dfnew.shape

## Map the density of the Asian population in the Twin Cities zip codes

In [ ]:
# download mn file
#!wget --quiet https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/mn_minnesota_zip_codes_geo.min.json
#Manually renamed to mn_geo.json    
with open('mn_geo.json') as json_data:
    mn_geo = json.load(json_data)

print('GeoJSON file loaded!')

In [ ]:
#Filter zip codes where population is > 10,000 Asians.  
df = dfnew[dfnew['Population'] > 10000]
df.to_csv('/Users/Mcair/Desktop/projects/DS_Code/ZipCodeAsian10000.csv',index=False)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#Import cleaned file if needed.
df = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/ZipCodeAsian10000.csv')
df['Population'].astype(str)
df['Population'] = df['Population'].astype('float64')
df['Zipcodes'] = df['Zipcodes'].astype('str')
df['Latitude'] = df['Latitude'].astype('float64')
df['Longitude'] = df['Longitude'].astype('float64')
df.head()

In [ ]:
# Initialize the map
tc = folium.Map(location=[44.9778, -93.2650], zoom_start=10)#,tiles='Mapbox Bright')
 
    # create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df['Population'].min(),
                              df['Population'].max(),
                              6, dtype=int)

threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Add the color for the chloropleth:
tc.choropleth(
 geo_data=mn_geo,
 data=df,
 columns=['Zipcodes', 'Population'],
 key_on='feature.properties.ZCTA5CE10',
 threshold_scale=threshold_scale,
 fill_color='BuPu',
 fill_opacity=0.8,
 line_opacity=0.1,
 legend_name='Population'
)
tc


In [ ]:
df.dtypes

## Use Yelp to search for Asian Grocery stores in the Twin Cities

In [ ]:
#Setup Yelp API variables
import requests

ClientID='OdcNPOaZPK-1Rw94P1wEMg'

api_key='8yezNH02SBHwHUnl4_GnP1KOlIjRy8pwV07jxGZxBf9dfL63nd0pe5jR_E8PmJ0UPQEkn0lYAUtvnH0cry-QfleuPP6bxqozijepm4EiIca_otHzhOxykFR4nR_4XHYx'

In [ ]:
SEARCH_LIMIT = 50 # limit of number of venues returned by Yelp API
#OFFSET=50
def getBusinesses(Zcodes, latudes, lotudes):
    
    biz_list = []
    #rad=500
    term = 'Asian Grocery'
    #term = ''
    
    for code, lat, lng in zip(Zcodes, latudes, lotudes):
    # create the API request URL      
        url = 'https://api.yelp.com/v3/businesses/search'     
        
        headers = {
        'Authorization': 'Bearer {}'.format(api_key),
        }
        
        url_params = {
                'term': term.replace(' ', '+'),
                'latitude': lat,
                'longitude': lng,
                'radius': 10000,
                'limit': SEARCH_LIMIT
        }
        # make the GET request
        results = requests.get(url, headers=headers, params=url_params).json()['businesses']
        
        # return only relevant information for each nearby venue
        biz_list.append([(
            code, 
            lat, 
            lng, 
            v['name'],
            v['location']['city'],
            v['location']['zip_code'],
            v['coordinates']['latitude'], 
            v['coordinates']['longitude']) for v in results])
    
   # nearby_biz = pd.DataFrame(pd.np.empty((0, 6)))    
    nearby_biz = pd.DataFrame([item for biz_list in biz_list for item in biz_list])
    nearby_biz.columns = ['Zipcodes', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Business',
                  'City', 
                  'Biz Zipcode',
                  'Biz Latitude', 
                  'Biz Longitude']
    
    return(nearby_biz)

In [ ]:
businesses = getBusinesses(Zcodes=df['Zipcodes'],
                               latudes=df['Latitude'],
                               lotudes=df['Longitude'])

In [ ]:
businesses.head()

In [ ]:
#Output to csv for further review and cleanup
businesses.to_csv('/Users/Mcair/Desktop/projects/DS_Code/tcbusinesses.csv')

In [ ]:
#Import the cleaned Twin Cities Asian Store file
Astores = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/tcbusiness1.csv')

In [ ]:
Astores['City'] = Astores['City'].astype('str')

In [ ]:
Astores.head()

In [ ]:
AStores = Astores.filter(['Business', 'Zipcodes.1','City','Biz Latitude', 'Biz Longitude'], axis=1)
AStores.rename({'Zipcodes.1': 'Zipcodes'}, axis=1, inplace=True)

In [ ]:
AStores.shape

In [ ]:
AStores.head()

In [ ]:
#drop duplicate reocords
AStores = AStores.drop_duplicates(subset=['Business', 'City','Biz Latitude', 'Biz Longitude'], keep='first')

In [ ]:
AStores

In [ ]:
AStores.shape

In [ ]:
AStores.dtypes

In [ ]:
#Export to file for manual upload if needed
AStores.to_csv('/Users/Mcair/Desktop/projects/DS_Code/AStores.csv',index=False)

## The following map illustrates the locations of established Asian markets and the density of Asians in the areas

In [ ]:
for lat, lng, name,cty, zipc in zip(AStores['Biz Latitude'], AStores['Biz Longitude'], AStores['Business'],AStores['City'],AStores['Zipcodes']):
    label = '{},{},{}'.format(name,cty,zipc)
    label = folium.Popup(label, parse_html=True)

    #folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='red')).add_to(tc)

    folium.CircleMarker([lat, lng],radius=5,popup=label,color='yellow',fill=True,fill_color='red',
                        fill_opacity=0.7,
                        parse_html=False).add_to(tc)  
tc

## Next we use the cordinates of the Asian markets to search for nearby venues using data from Foursquare.

In [ ]:
#setup Foursquare credentials for API calls
CLIENT_ID = 'CNH2BLPEMVP2JSUSH0RNO3TXIHTKVWSEFXIJ2VS4OM5CTMMO' # your Foursquare ID
CLIENT_SECRET = 'PHHVYR1MUJC2UKCRJXSSUFNFJMFIXHF45YDHLYK422QQE55R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(Zcodes, latudes, lotudes, radius=800):
    venues_list=[]
    for code, lat, lng in zip(Zcodes, latudes, lotudes):
    #print(name)     
    # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcodes', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
tcvenues = getNearbyVenues(Zcodes=AStores['Zipcodes'],
                           latudes=AStores['Biz Latitude'],
                           lotudes=AStores['Biz Longitude'])

In [ ]:
tcvenues.head()

In [ ]:
print(tcvenues.shape)
tcvenues.to_csv('/Users/Mcair/Desktop/projects/DS_Code/tcvenues.csv',index=False)

In [ ]:
tcvenues.head()

In [ ]:
tcvenues.shape

In [ ]:
#Import the tcvenues file if we do not need to requery Foursquare
tcvenues = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/tcvenues.csv')

In [ ]:
tcvenues.groupby('Zipcodes').count()

## Get venue counts for graph

In [ ]:
venuecount = tcvenues.groupby(['Zipcodes']).size().reset_index(name='Zipcodes')

In [ ]:
venuecount.head()

In [ ]:
venuecount.plot(kind='bar',x='Zipcodes',y='counts', title='Zip Code Venue Counts',width = .8,figsize=(10,8))


In [ ]:
print('There are {} uniques categories.'.format(len(tcvenues['Venue Category'].unique())))

In [ ]:
# one hot encoding
tc_onehot = pd.get_dummies(tcvenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tc_onehot['Zipcodes'] = tcvenues['Zipcodes'] 

# move neighborhood column to the first column
fixed_columns = [tc_onehot.columns[-1]] + list(tc_onehot.columns[:-1])
tc_onehot = tc_onehot[fixed_columns]

tc_onehot.head()

In [ ]:
tc_onehot.shape

In [ ]:
tc_grouped = tc_onehot.groupby('Zipcodes').mean().reset_index()
tc_grouped

In [ ]:
tc_grouped.shape

In [ ]:
num_top_venues = 5

for zipc in tc_grouped['Zipcodes']:
    print("----"+str(zipc)+"----")
    temp = tc_grouped[tc_grouped['Zipcodes'] == zipc].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

 ## Get the common venues around each of the established Asian Grocery Markets

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zipcodes']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Zipcodes'] = tc_grouped['Zipcodes']

for ind in np.arange(tc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

tc_grouped_clustering = tc_grouped.drop('Zipcodes', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tc_merged = df
#df['Zipcodes'] = df['Zipcodes'].astype('int64')
tcmerged = pd.merge(tc_merged, neighborhoods_venues_sorted.set_index('Zipcodes'), how='inner', on='Zipcodes')

In [ ]:
tc_merged.head()

In [ ]:
tcmerged.dtypes

In [ ]:
tcmerged['Zipcodes'] = tcmerged['Zipcodes'].astype('str')
tcmerged['Latitude'] = tcmerged['Latitude'].astype('float')
tcmerged['Longitude'] = tcmerged['Longitude'].astype('float')

tcmerged.drop(['State', 'National Rank'], axis = 1, inplace = True, errors = 'ignore')

In [ ]:
tcmerged#.head() # check the last columns!

In [ ]:
tcmerged.dtypes

In [ ]:
cdf = tcmerged[['Cluster Labels', '1st Most Common Venue']]

In [ ]:
cdf.head()

In [ ]:
cdf = pd.crosstab(cdf['Cluster Labels'],cdf['1st Most Common Venue'])

In [ ]:
cdf


## Common Venues in each cluster

In [ ]:



cdf.plot.bar(width = 1,figsize=(15,8),stacked=False)
plt.legend(title='Common Venues')

plt.show()

In [ ]:
xdfc = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/ZipCodeAsian10000.csv')

In [ ]:
xdfc['Zipcodes'] = xdfc['Zipcodes'].astype('str')

In [ ]:
# Initialize the map
clustermap = folium.Map(location=[44.9778, -93.2650], zoom_start=10)#,tiles='Mapbox Bright')
 
    # create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(xdfc['Population'].min(),
                              xdfc['Population'].max(),
                              6, dtype=int)

threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Add the color for the chloropleth:
clustermap.choropleth(
 geo_data=mn_geo,
 data=xdfc,
 columns=['Zipcodes', 'Population'],
 key_on='feature.properties.ZCTA5CE10',
 threshold_scale=threshold_scale,
 fill_color='BuPu',
 fill_opacity=0.8,
 line_opacity=0.1,
 legend_name='Population'
)


In [ ]:

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tcmerged['Latitude'], tcmerged['Longitude'], tcmerged['Zipcodes'], tcmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clustermap)
       
clustermap

## Cluster 0 - Asian Restuarants and Misc Venues

In [ ]:
tcmerged.loc[tcmerged['Cluster Labels'] == 0, tcmerged.columns[[0] + list(range(5, tcmerged.shape[1]))]]

## Cluster 1 - Rich in a variety of Venues: Ethinic Restaurants, Bars, Cafes, Groceries

In [ ]:
tcmerged.loc[tcmerged['Cluster Labels'] == 1, tcmerged.columns[[0] + list(range(5, tcmerged.shape[1]))]]

## Cluster 2 - Entertainment, C-Store, Liquor

In [ ]:
tcmerged.loc[tcmerged['Cluster Labels'] == 2, tcmerged.columns[[0] + list(range(5, tcmerged.shape[1]))]]

## Cluster 3 - Park, Asian Restaurant, Liquor

In [ ]:
tcmerged.loc[tcmerged['Cluster Labels'] == 3, tcmerged.columns[[0] + list(range(5, tcmerged.shape[1]))]]

## Create Map to illustrate where opportunities exists in relation to the current establishments

In [ ]:
#Import cleaned file to and get high density zipcode that don't have Asian Grocery Stores.
xdf = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/ZipCodeAsian10000.csv')


In [ ]:
xdf.shape

In [ ]:
xdf.head()

In [ ]:
xdf.dtypes

In [ ]:
#Add attribute to distinguish record dataset
xdf['in_xdf']='yes'

In [ ]:
xdf.head()

In [ ]:
xdf.drop(columns =['National Rank','State'], inplace = True) 

In [ ]:
xdf.shape

In [ ]:
#Import the current Asian stores
Astores = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/AStores.csv')

In [ ]:
#Add attribute to distinguish record dataset
Astores['inAstores']='yes'

In [ ]:
Astores.head()

In [ ]:
Astores.shape

In [ ]:
Astores.drop(columns =['Business','Biz Latitude', 'Biz Longitude', 'City'], inplace = True) 

In [ ]:
Astores.head()

In [ ]:
Astores = Astores.drop_duplicates()
#Astores = Astores.drop_duplicates(subset=['Business', 'City','Biz Latitude', 'Biz Longitude'], keep='first')


In [ ]:
Astores.shape

In [ ]:
#Join datasets
xdfx = pd.merge(Astores, xdf, how='right', on = 'Zipcodes')
#dfnew.drop(columns =["TCZipCodes"], inplace = True) 
xdfx.head()

In [ ]:
xdfx.shape


In [ ]:
xdfx['Zipcodes'] = xdfx['Zipcodes'].astype('str')

In [ ]:
xdfx.dtypes

In [ ]:
#Filter out zipcodes that already have Asian Stores
xdfx = xdfx.loc[(xdfx['inAstores'] != 'yes') & (xdfx['in_xdf'] == 'yes')]

In [ ]:
#There exist 71 Zip codes that have Asian Population of over 10K that do not have Asian Grocery Stores
xdfx.shape

In [ ]:
# Initialize the map
newst = folium.Map(location=[44.9778, -93.2650], zoom_start=10)#,tiles='Mapbox Bright')
 
    # create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(xdfx['Population'].min(),
                              xdfx['Population'].max(),
                              6, dtype=int)

threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Add the color for the chloropleth:
newst.choropleth(
 geo_data=mn_geo,
 data=xdfx,
 columns=['Zipcodes', 'Population'],
 key_on='feature.properties.ZCTA5CE10',
 threshold_scale=threshold_scale,
 fill_color='BuPu',
 fill_opacity=0.8,
 line_opacity=0.1,
 legend_name='Population'
)

In [ ]:
for lat, lng,cty, zipc in zip(xdfx['Latitude'], xdfx['Longitude'],xdfx['City'],xdfx['Zipcodes']):
    label = '{},{}'.format(cty,zipc)
    label = folium.Popup(label, parse_html=True)

    #folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='red')).add_to(tc)

    folium.CircleMarker([lat, lng],radius=8,popup=label,color='red',fill=False,
                        fill_opacity=0.7,parse_html=False).add_to(newst)  


In [ ]:
#Import the Astores file to map the current establishments.
AStores = pd.read_csv('/Users/Mcair/Desktop/projects/DS_Code/AStores.csv')

## Map of Zip Code Opportunities that can support new Asian Grocery Stores 

In [ ]:
for lat, lng, name,cty, zipc in zip(AStores['Biz Latitude'], AStores['Biz Longitude'], AStores['Business'],AStores['City'],AStores['Zipcodes']):
    label = '{},{},{}'.format(name,cty,zipc)
    label = folium.Popup(label, parse_html=True)

    #folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='red')).add_to(tc)

    folium.CircleMarker([lat, lng],radius=5,popup=label,color='yellow',fill=True,fill_color='red',
                        fill_opacity=0.7,
                        parse_html=False).add_to(newst)  
newst

> The red circles and popup represent the City and Zip Code where there is opportunity to setup shop as it visualizes the population desnsity of the zip code. The yellow/red circles are the current Asian grocery establishments. As you can see there are many zip codes that we can futher investigate.

> For example, the St. Paul zip code 55110 is has a high density of Asians but there are not Asian Grocery establishments